<a href="https://colab.research.google.com/github/tonguyenducmanh/PhoBERT-fine-tune/blob/main/PHOBERT_Excel_synonym_trainning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import (
    AutoModel,
    AutoTokenizer,
    get_linear_schedule_with_warmup
)
from torch.optim import AdamW

from torch import nn
from torch.utils.data import Dataset, DataLoader
import logging
from tqdm import tqdm


In [ ]:
# Thiết lập logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Đường dẫn đến model và dữ liệu
PHOBERT_PATH = "vinai/phobert-base-v2"
DATA_PATH = "/content/drive/MyDrive/data_train/train_data.csv"
OUTPUT_DIR = "/content/drive/MyDrive/data_train/ai_model/"
BATCH_SIZE = 16
MAX_LENGTH = 256
EPOCHS = 5
LEARNING_RATE = 2e-5

# Tạo thư mục output nếu chưa tồn tại
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [ ]:
# Định nghĩa model đồng nghĩa trái nghĩa
class SynonymModel(nn.Module):
    def __init__(self, bert_model):
        super(SynonymModel, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(0.1)
        self.hidden_size = self.bert.config.hidden_size
        self.classifier = nn.Linear(self.hidden_size * 3, 1)

    def mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] # [batch_size, seq_len, hidden_size]
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    def forward(self, input_ids1, attention_mask1, input_ids2, attention_mask2):
        outputs1 = self.bert(input_ids=input_ids1, attention_mask=attention_mask1)
        outputs2 = self.bert(input_ids=input_ids2, attention_mask=attention_mask2)

        # Mean pooling để lấy embedding cho mỗi câu
        embeddings1 = self.mean_pooling(outputs1, attention_mask1)
        embeddings2 = self.mean_pooling(outputs2, attention_mask2)

        # Kết hợp các embedding
        abs_diff = torch.abs(embeddings1 - embeddings2)
        combined_features = torch.cat([embeddings1, embeddings2, abs_diff], dim=1)

        # Classification head
        combined_features = self.dropout(combined_features)
        logits = self.classifier(combined_features)

        return logits

# Dataset cho bài toán so sánh đồng nghĩa
class SynonymDataset(Dataset):
    def __init__(self, encodings1, encodings2, labels):
        self.encodings1 = encodings1
        self.encodings2 = encodings2
        self.labels = labels

    def __getitem__(self, idx):
        item1 = {key: torch.tensor(val[idx]) for key, val in self.encodings1.items()}
        item2 = {key: torch.tensor(val[idx]) for key, val in self.encodings2.items()}

        item = {
            'input_ids1': item1['input_ids'],
            'attention_mask1': item1['attention_mask'],
            'input_ids2': item2['input_ids'],
            'attention_mask2': item2['attention_mask'],
            'labels': torch.tensor(self.labels[idx], dtype=torch.float)
        }
        return item

    def __len__(self):
        return len(self.labels)



In [ ]:
# Hàm chuẩn bị dữ liệu
def prepare_data(tokenizer):
    logger.info(f"Đang tải dữ liệu từ {DATA_PATH}")
    df = pd.read_csv(DATA_PATH)

    logger.info(f"Kích thước dữ liệu: {df.shape}")
    logger.info(f"Ví dụ dữ liệu:\n{df.head()}")

    # Chuyển đổi cột is_match từ chuỗi sang bool
    df['is_match'] = df['is_match'].astype(bool)

    # Tách dữ liệu thành tập huấn luyện và tập đánh giá
    train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

    logger.info(f"Kích thước tập huấn luyện: {train_df.shape}")
    logger.info(f"Kích thước tập đánh giá: {val_df.shape}")

    # Tokenize dữ liệu
    train_encodings1 = tokenizer(list(train_df['original_term']), truncation=True, padding='max_length', max_length=MAX_LENGTH)
    train_encodings2 = tokenizer(list(train_df['generated_term']), truncation=True, padding='max_length', max_length=MAX_LENGTH)
    train_labels = list(train_df['is_match'].astype(int))

    val_encodings1 = tokenizer(list(val_df['original_term']), truncation=True, padding='max_length', max_length=MAX_LENGTH)
    val_encodings2 = tokenizer(list(val_df['generated_term']), truncation=True, padding='max_length', max_length=MAX_LENGTH)
    val_labels = list(val_df['is_match'].astype(int))

    # Tạo dataset
    train_dataset = SynonymDataset(train_encodings1, train_encodings2, train_labels)
    val_dataset = SynonymDataset(val_encodings1, val_encodings2, val_labels)

    return train_dataset, val_dataset

# Hàm huấn luyện model
def train_model(model, train_dataloader, val_dataloader, optimizer, scheduler, device):
    best_val_accuracy = 0.0
    best_model_path = None

    logger.info("Bắt đầu huấn luyện model...")

    for epoch in range(EPOCHS):
        logger.info(f"Epoch {epoch + 1}/{EPOCHS}")

        # Training
        model.train()
        train_loss = 0.0
        train_preds = []
        train_true = []

        progress_bar = tqdm(train_dataloader, desc="Training")
        for batch in progress_bar:
            input_ids1 = batch['input_ids1'].to(device)
            attention_mask1 = batch['attention_mask1'].to(device)
            input_ids2 = batch['input_ids2'].to(device)
            attention_mask2 = batch['attention_mask2'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()

            outputs = model(input_ids1, attention_mask1, input_ids2, attention_mask2)

            loss_fn = nn.BCEWithLogitsLoss()
            loss = loss_fn(outputs.squeeze(), labels)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

            train_loss += loss.item()

            # Lưu dự đoán và nhãn cho đánh giá
            preds = torch.sigmoid(outputs.squeeze()).cpu().detach().numpy() >= 0.5
            train_preds.extend(preds.astype(int))
            train_true.extend(labels.cpu().detach().numpy().astype(int))

            progress_bar.set_postfix({'loss': loss.item()})

        avg_train_loss = train_loss / len(train_dataloader)
        train_accuracy = accuracy_score(train_true, train_preds)

        # Đánh giá trên tập validation
        model.eval()
        val_loss = 0.0
        val_preds = []
        val_true = []

        with torch.no_grad():
            for batch in tqdm(val_dataloader, desc="Validation"):
                input_ids1 = batch['input_ids1'].to(device)
                attention_mask1 = batch['attention_mask1'].to(device)
                input_ids2 = batch['input_ids2'].to(device)
                attention_mask2 = batch['attention_mask2'].to(device)
                labels = batch['labels'].to(device)

                outputs = model(input_ids1, attention_mask1, input_ids2, attention_mask2)

                loss_fn = nn.BCEWithLogitsLoss()
                loss = loss_fn(outputs.squeeze(), labels)

                val_loss += loss.item()

                preds = torch.sigmoid(outputs.squeeze()).cpu().numpy() >= 0.5
                val_preds.extend(preds.astype(int))
                val_true.extend(labels.cpu().numpy().astype(int))

        avg_val_loss = val_loss / len(val_dataloader)
        val_accuracy = accuracy_score(val_true, val_preds)
        precision, recall, f1, _ = precision_recall_fscore_support(val_true, val_preds, average='binary')

        logger.info(f"Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
        logger.info(f"Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")
        logger.info(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

        # Lưu model tốt nhất
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            model_to_save = model.module if hasattr(model, 'module') else model
            best_model_path = os.path.join(OUTPUT_DIR, f"best_model_epoch_{epoch+1}.pt")

            logger.info(f"Lưu model tốt nhất với độ chính xác: {best_val_accuracy:.4f}")
            torch.save({
                'epoch': epoch,
                'model_state_dict': model_to_save.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_accuracy': best_val_accuracy,
            }, best_model_path)

    return best_model_path

# Hàm dự đoán đồng nghĩa
def predict_synonyms(model, tokenizer, term1, term2, device):
    model.eval()

    # Tokenize hai term
    encodings1 = tokenizer(term1, truncation=True, padding='max_length', max_length=MAX_LENGTH, return_tensors='pt')
    encodings2 = tokenizer(term2, truncation=True, padding='max_length', max_length=MAX_LENGTH, return_tensors='pt')

    input_ids1 = encodings1['input_ids'].to(device)
    attention_mask1 = encodings1['attention_mask'].to(device)
    input_ids2 = encodings2['input_ids'].to(device)
    attention_mask2 = encodings2['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids1, attention_mask1, input_ids2, attention_mask2)

    probability = torch.sigmoid(outputs.squeeze()).item()
    is_synonym = probability >= 0.5

    return is_synonym, probability

def main():
    # Kiểm tra GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logger.info(f"Sử dụng thiết bị: {device}")

    # Tải tokenizer và model từ đường dẫn cục bộ
    logger.info(f"Tải PhoBERT model từ {PHOBERT_PATH}")
    try:
        tokenizer = AutoTokenizer.from_pretrained(PHOBERT_PATH)
        bert_model = AutoModel.from_pretrained(PHOBERT_PATH)
        logger.info("Tải model thành công")
    except Exception as e:
        logger.error(f"Lỗi khi tải model: {str(e)}")
        logger.error("Hãy đảm bảo đã copy PhoBERT model vào thư mục vinai_phobert/ trước khi chạy script này")
        return

    # Tải dữ liệu
    train_dataset, val_dataset = prepare_data(tokenizer)

    # Tạo DataLoader
    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

    # Khởi tạo model
    model = SynonymModel(bert_model)
    model.to(device)

    # Khởi tạo optimizer và scheduler
    optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
    total_steps = len(train_dataloader) * EPOCHS
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )

    # Huấn luyện model
    best_model_path = train_model(model, train_dataloader, val_dataloader, optimizer, scheduler, device)

    # Tải model tốt nhất
    if best_model_path:
        logger.info(f"Tải model tốt nhất từ {best_model_path}")
        checkpoint = torch.load(best_model_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])

        # Test dự đoán với một vài ví dụ
        test_examples = [
            ("Áp dụng chiết khấu", "Giảm giá"),
            ("Mã khách hàng", "Mã hàng"),
        ]

        logger.info("Kiểm tra kết quả dự đoán:")
        for term1, term2 in test_examples:
            is_synonym, probability = predict_synonyms(model, tokenizer, term1, term2, device)
            logger.info(f"'{term1}' và '{term2}': {'là đồng nghĩa' if is_synonym else 'không là đồng nghĩa'} (xác suất: {probability:.4f})")

In [ ]:
main()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

Training:   0%|          | 12/2802 [00:15<1:00:38,  1.30s/it, loss=0.324]


KeyboardInterrupt: 

In [ ]:
# Tải model tốt nhất đã lưu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"Sử dụng thiết bị: {device}")

# Tải tokenizer và model từ đường dẫn cục bộ
logger.info(f"Tải PhoBERT model từ {PHOBERT_PATH}")
try:
    tokenizer = AutoTokenizer.from_pretrained(PHOBERT_PATH)
    bert_model = AutoModel.from_pretrained(PHOBERT_PATH)
    logger.info("Tải model thành công")
except Exception as e:
    logger.error(f"Lỗi khi tải model: {str(e)}")
    logger.error("Hãy đảm bảo đã copy PhoBERT model vào thư mục vinai_phobert/ trước khi chạy script này")

model = SynonymModel(bert_model)
model.to(device)

best_model_path = os.path.join(OUTPUT_DIR, "best_model_epoch_1.pt") # Sử dụng model đã lưu ở epoch 1
if os.path.exists(best_model_path):
    logger.info(f"Tải model tốt nhất từ {best_model_path}")
    checkpoint = torch.load(best_model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    logger.info("Tải state dict model thành công")
else:
    logger.error(f"Không tìm thấy model tại {best_model_path}. Vui lòng đảm bảo model đã được lưu thành công.")


# Test dự đoán với một vài ví dụ mới
test_examples_new = [
    ("Máy tính xách tay", "Laptop"),
    ("Điện thoại di động", "Máy tính"),
    ("Xe hơi", "Ô tô"),
    ("Nhà", "Căn hộ"),
    ("Buồn bã", "Vui vẻ"),
    ("Mã khách hàng", "Mã KH"),
    ("Mã khách hàng", "Mã hàng")
]

print("Kiểm tra kết quả dự đoán với các ví dụ mới:")
for term1, term2 in test_examples_new:
    if 'model' in locals() and 'tokenizer' in locals():
        is_synonym, probability = predict_synonyms(model, tokenizer, term1, term2, device)
        print(f"'{term1}' và '{term2}': {'là đồng nghĩa' if is_synonym else 'không là đồng nghĩa'} (xác suất: {probability:.4f})")
    else:
        logger.error("Model hoặc tokenizer chưa được tải.")

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Kiểm tra kết quả dự đoán với các ví dụ mới:
'Máy tính xách tay' và 'Laptop': là đồng nghĩa (xác suất: 0.6284)
'Điện thoại di động' và 'Máy tính': không là đồng nghĩa (xác suất: 0.1758)
'Xe hơi' và 'Ô tô': là đồng nghĩa (xác suất: 0.5899)
'Nhà' và 'Căn hộ': là đồng nghĩa (xác suất: 0.6426)
'Buồn bã' và 'Vui vẻ': không là đồng nghĩa (xác suất: 0.1235)
'Mã khách hàng' và 'Mã KH': là đồng nghĩa (xác suất: 0.7481)
'Mã khách hàng' và 'Mã hàng': là đồng nghĩa (xác suất: 0.6653)
